# Enron Data ETL

In [18]:
# Dependencies
from sqlalchemy.sql import func
from typing import List
from sqlalchemy.orm import relationship, sessionmaker, Mapped, mapped_column
from sqlalchemy import create_engine, func, and_, or_, MetaData, Table, Column, Integer, String, text, desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.exc import SQLAlchemyError
from config import user_name, user_password
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.parse

from warnings import filterwarnings
filterwarnings('ignore')

In [19]:
encoded_password = urllib.parse.quote_plus(user_password)
print(encoded_password)  # 输出: ifeng%408276

ifeng%408276


In [20]:
# Connect mysql DB with Sqlalchemy orm
host = 'localhost'
port = 3306
database = 'Enron_DB'

try:
    engine = create_engine(f'mysql+pymysql://{user_name}:{encoded_password}@{host}:{port}/{database}')
    connection = engine.connect()
    print("Connection successful")
    connection.close()
except SQLAlchemyError as e:
    print(f"Connection fail: {e}")

Connection successful


In [21]:
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [22]:
# Crate metadata object
metadata = MetaData()
# Reflect all table in the database
metadata.reflect(bind=engine)

# display the all table name
for table_name in metadata.tables.keys():
    print(table_name)

employeelist
message
recipientinfo
referenceinfo


In [23]:
# Query employee_abc 
message = Table('message', metadata, autoload=True, autoload_with=engine)
for cols in message.columns:
    print(cols.name)

mid
sender
date
message_id
subject
body
folder


In [ ]:
# query time, cnt_letter from 1999 to 2022
results = (
    session.query(
        message.date,
        func.count(message.mid).label('Msg_CNT')
    )
    .filter(message.date.between(dt.date(1998, 1, 1), dt.date(2002, 12, 31))
    # .filter(extract('year', message.date).between(1998, 2002)) 
    .group_by(message.date)
    .order_by(message.date)
    .all()
)

letter_data = pd.DataFrame(results, columns=['date', 'Msg_CNT'])